# For colab environment

In [2]:
import torch 
print(torch.cuda.is_available())
print(torch.__version__)

True
2.6.0+cu124


In [3]:
# !git clone https://github.com/BernardMoy/NLP-PCL-Classification.git

In [4]:
# %cd NLP-PCL-Classification/

In [5]:
!nvidia-smi

Sat Feb 28 17:14:50 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA TITAN Xp                Off |   00000000:02:00.0  On |                  N/A |
| 23%   28C    P8             18W /  250W |     191MiB /  12288MiB |     33%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Load train and validation data set

In [6]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt
from collections import Counter

df = pd.read_csv('data/dontpatronizeme_pcl.tsv', sep='\t')

# Remove rows with NA labels 
df = df.dropna() 

# Add a bool_labels column for binary classification
df["bool_labels"] = df["label"] > 1   # is PCL if >1

# train val split 
train_labels = pd.read_csv('data/train_semeval_parids-labels.csv')["par_id"]
val_labels = pd.read_csv('data/dev_semeval_parids-labels.csv')["par_id"]
df_train = df[df["par_id"].isin(train_labels)]
df_val = df[df["par_id"].isin(val_labels)]


# Tokenization

In [7]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AutoConfig, Trainer, TrainingArguments

tokenizer = RobertaTokenizer.from_pretrained("roberta-base") 

# Create text with contextual information 
def tokenize(df): 
    text_with_context = df["text"] # df["keyword"] + "</s><s>" + df["country_code"] + "</s><s>" + df["text"]

    encoding = tokenizer(
        text_with_context.tolist(), 
        padding="max_length",   # Add padding to shorter sentences 
        max_length=256,
        truncation = True, 
        return_attention_mask = True 
    )

    return encoding

/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


# Convert to pyTorch dataset

In [8]:
import torch 
from torch.utils.data import DataLoader, TensorDataset
from datasets import Dataset

def to_dataset(df): 
    # Obtain tokens (input_ids, attention_mask) from the dataset 
    encoding = tokenize(df) 

    # Return huggingface dataset 
    return Dataset.from_dict({
        "input_ids": encoding["input_ids"], 
        "attention_mask": encoding["attention_mask"], 
        "label": df["bool_labels"].values 
    })

In [9]:
train_dataset = to_dataset(df_train)
val_dataset = to_dataset(df_val) 

# set to torch format 
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Training 

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)

    # Calculate metrics 
    accuracy = accuracy_score(labels, predictions) 
    precision = precision_score(labels, predictions) 
    recall = recall_score(labels, predictions) 
    f1 = f1_score(labels, predictions) 

    return {
        "accuracy": accuracy, 
        "precision": precision, 
        "recall": recall, 
        "f1": f1 
    }


In [12]:
# Load roberta sequence classification model 
config = AutoConfig.from_pretrained("roberta-base", num_labels=2)  # Binary classification
model = RobertaForSequenceClassification.from_pretrained("roberta-base", config = config)

BATCH_SIZE = 32

# Set up training arguments 
training_args = TrainingArguments(
    fp16=True, 
    num_train_epochs=5, 
    learning_rate=2e-5, 
    weight_decay=0.01,
    warmup_steps=500, 
    save_strategy="no",  # low disk space 
    load_best_model_at_end=True, 
    logging_steps=50,
    output_dir="./predictions", 
    evaluation_strategy="no", 
    per_device_eval_batch_size=BATCH_SIZE, 
    per_device_train_batch_size=BATCH_SIZE, 
)

# Set up trainer 
trainer = Trainer(
    model = model, 
    args = training_args, 
    train_dataset=train_dataset, 
    eval_dataset=val_dataset, 
    compute_metrics=compute_metrics
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/accelerate/accelerator.py:439: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [13]:
trainer.train() 

  0%|          | 0/1310 [00:00<?, ?it/s]

{'loss': 0.7528, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.19}
{'loss': 0.3781, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.38}
{'loss': 0.3253, 'learning_rate': 6e-06, 'epoch': 0.57}
{'loss': 0.2623, 'learning_rate': 7.960000000000002e-06, 'epoch': 0.76}
{'loss': 0.2316, 'learning_rate': 9.960000000000001e-06, 'epoch': 0.95}
{'loss': 0.2213, 'learning_rate': 1.196e-05, 'epoch': 1.15}
{'loss': 0.2217, 'learning_rate': 1.392e-05, 'epoch': 1.34}
{'loss': 0.1928, 'learning_rate': 1.5920000000000003e-05, 'epoch': 1.53}
{'loss': 0.2119, 'learning_rate': 1.792e-05, 'epoch': 1.72}
{'loss': 0.193, 'learning_rate': 1.9920000000000002e-05, 'epoch': 1.91}
{'loss': 0.1722, 'learning_rate': 1.8814814814814816e-05, 'epoch': 2.1}
{'loss': 0.1606, 'learning_rate': 1.7580246913580247e-05, 'epoch': 2.29}
{'loss': 0.1429, 'learning_rate': 1.6345679012345682e-05, 'epoch': 2.48}
{'loss': 0.1653, 'learning_rate': 1.5111111111111112e-05, 'epoch': 2.67}
{'loss': 0.1602, 'learning_rate': 1.387

TrainOutput(global_step=1310, training_loss=0.17404840074422706, metrics={'train_runtime': 989.5359, 'train_samples_per_second': 42.318, 'train_steps_per_second': 1.324, 'train_loss': 0.17404840074422706, 'epoch': 5.0})

In [14]:
trainer.evaluate()

  0%|          | 0/66 [00:00<?, ?it/s]

{'eval_loss': 0.36206522583961487,
 'eval_accuracy': 0.9235547061634019,
 'eval_precision': 0.6065573770491803,
 'eval_recall': 0.5577889447236181,
 'eval_f1': 0.581151832460733,
 'eval_runtime': 17.6719,
 'eval_samples_per_second': 118.436,
 'eval_steps_per_second': 3.735,
 'epoch': 5.0}